In [1]:
INPUT_FILE = 'app.db'

In [2]:
import sqlite3
from pathlib import Path

with sqlite3.connect(Path(INPUT_FILE)) as db:
    db_cursor = db.cursor()
    text_cursor = db_cursor.execute('SELECT id FROM import_text')
    docs = text_cursor.fetchall()

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(docs, train_size=0.7)

Update training dataset

In [12]:
from tqdm import tqdm

def update_dataset(ids, table_name):
	with sqlite3.connect(Path(INPUT_FILE)) as db:
		db_cursor = db.cursor()
		# first, remove question set
		db_cursor.execute(f'DELETE FROM {table_name}')
  
		for doc_id in tqdm(ids):
			questions_cursor = db_cursor.execute('SELECT id FROM import_questions WHERE import_text_id = ?', doc_id)
			doc_questions_ids = questions_cursor.fetchall()
	
			db_cursor.executemany(f'INSERT INTO {table_name} VALUES (?)', doc_questions_ids)
   

In [13]:
update_dataset(train, 'train_dataset')

100%|██████████| 13234/13234 [01:22<00:00, 160.68it/s]


Update testing dataset

In [14]:
update_dataset(test, 'test_dataset')

100%|██████████| 5672/5672 [00:34<00:00, 164.44it/s]
